# AI Assisted Annotation with OHIF

Zero foot print Dicom viewers are ideal for radiology workflow. 
[Open Health Imaging Foundation (OHIF)](http://ohif.org/) Viewer is an open source, web-based, medical imaging viewer. 
It can be configured to connect to Image Archives that support DicomWeb, 
and offers support for mapping to proprietary API formats. 
OHIF maintained extensions add support for viewing, annotating, 
and reporting on DICOM images in 2D (slices) and 3D (volumes). 
OHIF by it self is a web viewer that requires a PACS on the back end to hold the dicom images. 
You can use either [Dcm4chee](https://github.com/dcm4che/dcm4chee-arc-light/wiki) or [Orthanc](https://www.orthanc-server.com/). 

For this we expand on how to use OHIF with AIAA. 
This note book will guide your through how to set up 
- OHIF web viewer with Nvidia AIAA integration 
- Orthanc
- This notebooks 


## Prerequisites
- Nvidia GPU with 8Gb of memory
- Going through [AIAA Notebook](AIAA.ipynb) to get basic understanding of AIAA.    
- For trouble shot, basic understanding of docker and docker compose is recommended 


## 1. Setup OHIF 
Nvidia has integrated AIAA into OHIF as a plugin. 
We have packaged all under OHIF-Orthanc sub-folder. 
1. Launch OHIF with Orthanc or DCM4Chee
    1. Open terminal on your bare bone host
    2. cd into `OHIF-Orthanc` sub-folders
    3. (optional) edit docker-compose.yml to change/ map the data dir. 
    By default it will be inside the sub-folder
    4. Run `./restart.sh`. This will launch this notebooks, OHIF, and orthanc
    5. Basic authentication is provided. User/password is `clara/clara`  
2. Check things are running: 
    1. OHIF: </br> 
    Go to your browser [localhost:3030]() to see OHIF web page as below
    <br><img src="screenShots/OHIFList.png" alt="Drawing" style="height: 300px;"/><br> 
    2. Orthanc: </br> 
    Go to your browser at [localhost:3030/pacs-admin/]() to see Orthanc web UI (username/password is orthanc/orthanc) 
    <br><img src="screenShots/Orthanc.png" alt="Drawing" style="height: 300px;"/><br> 
    3. This Notebooks: </br>
    Go to your browser at [localhost:3030/notebooks/]() 
    and use the token printed out after running the `restart.sh` script

## 2. Download Dicom Studies

OHIF and the back end PACS uses dicom format. 
For this you need to have a some Dicom studies locally.
You can run [TCIA Notebook](../Data/tcia-master/TCIADownloader.ipynb) 
with any of the sample data links. 
You should find the downloaded data under `/claraDevDay/Data/<Dataname>/DCM/`  


### 2.1 (optional) Convert Nifti to Dicom 


In [ ]:
Data_ROOT="/claraDevDay/Data/"
AIAA_ROOT="/claraDevDay/AIAA/"

Install xmedcon 

In [ ]:
!apt-get install -y xmedcon


You can see all features of the medcon by running `medcon -h`. 
Cell below will convert the sample spleen dataset in to dicom. 
Then add some date/time and fake dicom metadata in order to make it a valide Dicom 

In [ ]:
!medcon -c dicom -f $Data_ROOT/sampleData/imagesTr/spleen_8.nii.gz -o $AIAA_ROOT/dicomSample/spleen_8.dcm
!$AIAA_ROOT/OHIF/dcm4Chee/upload_dicom.sh $AIAA_ROOT/dicomSample/ clarasa-station
!python fixDicom.py


## 3. Upload a dicom study into the PACS (orthanc)
Now that you have dicom images, 
we would want to upload it to our PACS. 
For this we can use dcmtk library  

In [ ]:
# install apt-get install dcmtk
!apt-get -y install dcmtk

As we are using docker compose internal network we can directly upload to orthanc using its intenal name 
`orthanc` and its internal port 4242 

In [ ]:
!storescu -v +sd +r -xb -v -aet "ORTHANC" orthanc 4242 /claraDevDay/Data/NSCLC_5/DCM/

## 4. Open OHIF and use AIAA
Now that we have dicom data in our PACS, you can open OHIF in your browser and click on a patient to view it as below.
<br><img src="screenShots/OHIF.png" alt="Drawing" style="height: 300px;"/><br> 


Next you need to setup the AIAA server url as `http://<yourmachineIP>:5000` click refresh to see the models you have loaded. 
You should see the spleen segmentation and the deep grow models loaded.


## 5. Stop All
To stop all dockers (Notebooks, OHIF and PACS) you should run `./stop.sh`. 
***Note: Running cell below would terminate this notebook***


In [ ]:
#Note: Running cell below would terminate this notebook
#!./stop.sh`
